# CONNECTING DATABASE TO GUI

### IMPORTING REQUIRED THINGS

In [23]:
import tkinter as tk
from tkinter import messagebox
import sqlite3

### CREATING TABLE

In [39]:
conn = sqlite3.connect("students.db")   
cursor = conn.cursor()   #a cursor as a finger that helps you point at, read, or write information in a database                                              

# Create "students" table if not exists
cursor.execute("""
CREATE TABLE IF NOT EXISTS students (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    age INTEGER NOT NULL,
    course TEXT NOT NULL,
    gender TEXT NOT NULL,
    activity INTEGER NOT NULL
)
""")
conn.commit()  #saves the data                            
conn.close()  # close the connection

### GUI WINDOW

In [ ]:
root = tk.Tk()    # creating the instance of Tk
root.title("Student Registration") # giving title to gui window
root.geometry("400x400")   #window size

#NAME
tk.Label(root, text="Name:").pack(anchor="w", padx=10)   # giving label
name_entry = tk.Entry(root)            # taken input and store in 'name_entry'
name_entry.pack(anchor="w", padx=10)    #pack used to display data , anchor tell us where we put data llike w= west likewise

# Age Entry
tk.Label(root, text="Age:").pack(anchor="w", padx=10)
age_entry = tk.Entry(root, width=30)
age_entry.pack(anchor="w", padx=10)

# Dropdown Menu (Course Selection)
course_var = tk.StringVar()                      #defining a variable with its type STringVAR
tk.Label(root, text="Course:").pack(anchor="w", padx=10)  

course_var.set(courses[0])         # default value will be course[0] which is Math
courses = ["Math", "Science", "Arts", "History"]
dropdown = tk.OptionMenu(root, course_var, *courses)  # course_var stores value to be displayed (default 'math') and *courses * divide course in se
dropdown.pack(anchor="w", padx=10)

# Radio Buttons (Gender Selection)
var = tk.StringVar(value="Male")  # Default value also defining var datatype
tk.Label(root, text="Gender:").pack(anchor="w", padx=10)
tk.Radiobutton(root, text="Male", variable=var, value="Male").pack(anchor="w", padx=15)
tk.Radiobutton(root, text="Female", variable=var, value="Female").pack(anchor="w", padx=15)

# Checkbox (Extra Activities)
activity_var = tk.IntVar()
tk.Checkbutton(root,text="takepart in Extra Activities",variable=activity_var).pack(anchor="w",padx=10,pady=5)# variable=activity_var data going to db

# Buttons
tk.Button(root, text="Save Data", command=save_data, width=15).pack(anchor="w", padx=10, pady=5)
tk.Button(root, text="Show Data", command=show_data, width=15).pack(anchor="w", padx=10, pady=5)

# Listbox to Display Data
tk.Label(root, text="Student Records:").pack(anchor="w", padx=10, pady=1)
listbox = tk.Listbox(root, width=50, height=5)
listbox.pack(anchor="w", pady=15, padx=10)

# Run the GUI
root.mainloop()

### CONNECTION OF GUI AND DB || CODE TO SHOW DATA

In [55]:
# Function to Save Data
def save_data():                   
    name = name_entry.get()
    age = age_entry.get()
    course = course_var.get()
    gender = var.get()
    activity = activity_var.get()  # where activity_var is variable name given while taking input from gui

    if name and age.isdigit():  # Ensure name is entered and age is a number
        conn = sqlite3.connect("students.db")
        cursor = conn.cursor()

        # Save data into the database
        cursor.execute("INSERT INTO students (name, age, course, gender, activity) VALUES (?, ?, ?, ?, ?)",
                       (name, int(age), course, gender, activity))
        conn.commit()
        conn.close()

        messagebox.showinfo("Success", "Data saved successfully!") # where success is title of msgbox and other is message

        # Clear input fields
        name_entry.delete(0, tk.END)              #0 is initial value and tk is end value (its deleting complete value from start to end
        age_entry.delete(0, tk.END)

        show_data()  # Refresh the Listbox
    else:
        messagebox.showerror("Error", "Please enter a valid Name and Age.")

# Function to Show Data
def show_data():
    listbox.delete(0, tk.END)  # Clear existing data from start to end
    conn = sqlite3.connect("students.db")
    cursor = conn.cursor()
    cursor.execute("SELECT name, age, course, gender, activity FROM students")
    for row in cursor.fetchall():
        listbox.insert(tk.END, f"{row[0]}, Age: {row[1]}, {row[2]}, {row[3]}, Extra: {'Yes' if row[4] else 'No'}")
    conn.close()
